Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропущенных значений нет. Это хорошо.

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** не изменится

**Обоснование:** в исходной задаче модель линейной регрессии обучается на основе формулы $a=Xw$, где $a$ - предсказания модели, $X$ - объекты, $w$ - функция потерь. При обучении модели с преобразованными данными же появляется еще один множитель - $P$ - случайная обратимая матрица, с помощью которой мы кодируем данные от злоумышленников. Тогда формула расчета предсказаний выглядит следующим образом: $a'=XPw$.

Получается, что данные о каждом человеке умножаются на вектор. Модель получит на вход другие числа, но корреляции между признаками, веса и т.д остаются прежними.

## 3. Алгоритм преобразования

**Алгоритм**

$a_2 = XP((XP)^T XP)^{-1} (XP)^T y$

Раскроем скобки при знаках транспонирования:

$a_2 = XP(P^T X^T XP)^{-1} P^T X^T y$

Раскроем скобку $(P^T X^T XP)^{-1}$. Т.к $P$, $X^T X$, $P^T$ - квадратные и обратимые матрицы, то воспользуемся свойством: $(AB)^{-1} = B^{-1} A^{-1}$:

$a_2 = XP(X^T XP)^{-1} (P^T)^{-1}P^TX^T y$

Воспользуемся еще одним свойством матриц: $A A^{-1} = E$

$a_2 = XP(X^T XP)^{-1} EX^T y$

$E$ - единичная матрица, которая не меняет данные. Вынесем $P$ из многочлена $(X^T XP)$:

$a_2 = XPP^{-1}(X^T X)^{-1} X^T y$

Тогда:

$a_2 = X(X^T X)^{-1} X^T y$

Но $а_1 = X(X^T X)^{-1} X^T y$

Следовательно  $a_2 = a_1$. 

Что и требовалось доказать.

**Обоснование**

Получили, что при умножении исходной матрицы признаков на любую другую обратимую матрицу, мы совершаем равносильные преобразования. Следовательно такой метод кодирования информации подходит в данной задаче

## 4. Проверка алгоритма

Умножим датасет на матрицу P. Но для начала проверим, что она обратимая

In [4]:
P = (np.random.normal(size=[4, 4]))
np.linalg.inv(P) 

array([[-3.51163372e-01, -7.46778282e-01,  5.28587178e-02,
        -6.23631782e-01],
       [ 1.22707514e-01,  9.08577835e-01, -2.21454615e-01,
        -8.17905331e-05],
       [-9.47088232e-02,  1.18514800e+00, -6.87413743e-01,
         1.09626787e+00],
       [-2.07311666e-02, -3.14173397e-01,  6.32656290e-01,
        -3.95610432e-01]])

Матрица обратимая, можем приступать к умножению

In [5]:
P = (np.random.normal(size=[4, 4]))
df_new = df.drop('Страховые выплаты', axis=1) @ P
df_new['4'] = df['Страховые выплаты']
df_new.columns = ['0', '1', '2', '3', '4']
df_new

,0,1,2,3,4
0,12307.282287,9406.818342,20314.675979,-44292.629554,0
1,9448.033323,7184.955464,15561.898934,-33941.787175,1
2,5226.265334,3965.777132,8599.686241,-18759.092526,0
3,10327.974130,7929.952082,17080.866218,-37228.916144,0
4,6485.353418,4939.560253,10688.886260,-23311.679144,0
...,...,...,...,...,...
4995,8855.361515,6772.633205,14621.728710,-31878.752436,0
4996,12988.774285,9954.909049,21463.174446,-46785.187434,0
4997,8399.962330,6441.453865,13885.364651,-30267.375334,0
4998,8106.655166,6205.467826,13392.742295,-29200.384791,0


Выделим целевой признак и остальные

In [6]:
feature = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']
print(feature.shape, target.shape)

(5000, 4) (5000,)


Разделим каждый из 2-х датасетов на 2 выборки - train для обучения модели, test - для вычисления метрики 

In [7]:
feature_train, feature_test, target_train, target_test = train_test_split(feature, target,
                                                                          random_state=12345, test_size=0.25)
print(feature_train.shape, target_train.shape, feature_test.shape, target_test.shape)

(3750, 4) (3750,) (1250, 4) (1250,)


In [8]:
new_feature = df_new.drop('4', axis=1)
new_target = df_new['4']
print(new_feature.shape, new_target.shape)

(5000, 4) (5000,)


In [9]:
new_feature_train, new_feature_test, new_target_train, new_target_test = train_test_split(new_feature, new_target,
                                                                          random_state=12345, test_size=0.25)
print(new_feature_train.shape, new_target_train.shape, new_feature_test.shape, new_target_test.shape)

(3750, 4) (3750,) (1250, 4) (1250,)


Обучим модели и посмотрим изменится ли метрика R2

In [10]:
model = LinearRegression()

In [11]:
model.fit(feature_train, target_train)
predict = model.predict(feature_test)

In [12]:
new_model = LinearRegression()

In [13]:
new_model.fit(new_feature_train, new_target_train)
new_predict = new_model.predict(new_feature_test)

In [14]:
r2 = r2_score(predict, target_test)
r2_new = r2_score(new_predict, new_target_test)
print('R2 до преобразования:', r2)
print('R2 после преобразования:', r2_new)

R2 до преобразования: -0.2869270212348376
R2 после преобразования: -0.28692702123478253


Как мы можем заметить, значение метрики R2 не изменилось, а значит и кодирование данных путем перемножениям матриц имеет место быть.

Попробуем восстановить закодированные данные

In [15]:
display(np.round(df_new.drop('4', axis=1) @ np.linalg.inv(P)))
display(df)

,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,-0.0,46.0,38000.0,1.0
2,-0.0,29.0,21000.0,0.0
3,-0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,0.0
...,...,...,...,...
4995,-0.0,28.0,35700.0,2.0
4996,-0.0,34.0,52400.0,1.0
4997,-0.0,20.0,33900.0,2.0
4998,1.0,22.0,32700.0,3.0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


Как мы можем заметить, данные можно восстановить, имея матрицу, с помощью которой мы кодировали данные.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [X]  Выполнен шаг 1: данные загружены
- [X]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [X]  Указан правильный вариант ответа
    - [X]  Вариант обоснован
- [X]  Выполнен шаг 3: предложен алгоритм преобразования
    - [X]  Алгоритм описан
    - [X]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования